\
            # 48. 系统设计：缓存（Cache）（Caching Design）

            目标：把缓存“用对且用稳”：选型、key 设计、TTL/失效、缓存一致性、热点与雪崩/穿透/击穿。
本章用 Python 标准库实现一个 TTLCache，并演示 cache-aside 与“击穿”防护。

            > 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行（第三方依赖会做可选降级）。


## 前置知识

- 字典/函数/异常
- 并发基础（线程）更佳


## 知识点地图

- 1. 缓存放在哪里：客户端/CDN/反向代理/应用内/分布式
- 2. 缓存模式：cache-aside / read-through / write-through（核心）
- 3. Key 设计：命名空间、版本、参数归一化
- 4. TTL 与失效：一致性与成本的权衡
- 5. 三大故障：穿透/击穿/雪崩 与对策
- 6. 可运行：实现一个 TTLCache（最小）
- 7. cache-aside 示例：读回填 + 写删缓存
- 8. 击穿防护：singleflight（互斥回源）


## 自检清单（学完打勾）

- [ ] 知道缓存适合解决什么问题（延迟/吞吐/成本）以及代价（不一致/复杂度）
- [ ] 掌握常见缓存模式：cache-aside/read-through/write-through
- [ ] 会设计 cache key（命名空间、版本、参数归一化）
- [ ] 理解 TTL、失效策略与一致性权衡
- [ ] 知道雪崩/穿透/击穿的现象与常见解决方案


In [ ]:
\
from pathlib import Path

ART = Path('_nb_artifacts')
ART.mkdir(exist_ok=True)
print('artifacts dir:', ART.resolve())


## 知识点 1：缓存放在哪里：客户端/CDN/反向代理/应用内/分布式

常见层次：
- 浏览器缓存（Cache-Control/ETag）
- CDN 缓存（静态资源/边缘缓存）
- 反向代理缓存（Nginx/Varnish）
- 应用内存缓存（进程内 dict/LRU）
- 分布式缓存（Redis/Memcached）

选择原则：离用户越近越快，但失效与一致性更难。


## 知识点 2：缓存模式：cache-aside / read-through / write-through（核心）

- cache-aside（旁路缓存）：
  - 读：先查缓存，miss 再查 DB 并回填缓存
  - 写：先写 DB，再删除/更新缓存
  - 优点：简单通用；缺点：一致性要设计好

- read-through：应用只读缓存，缓存 miss 时自动回源（由缓存层实现）
- write-through：写入先到缓存，再同步写 DB
- write-back：写缓存即可，异步落 DB（吞吐高但风险大）


## 知识点 3：Key 设计：命名空间、版本、参数归一化

Key 设计决定缓存命中率与可维护性：
- 命名空间：`user:profile:{user_id}`
- 版本：`v1:user:profile:{user_id}`（schema/序列化变化时可整体切换）
- 参数归一化：排序/去默认值，避免同一语义产生多个 key
- 颗粒度：太细 key 爆炸，太粗命中低


## 知识点 4：TTL 与失效：一致性与成本的权衡

- TTL：让数据“过一段时间自动失效”，降低永久不一致风险。
- 失效策略：
  - 主动删除（写 DB 后删 cache）
  - 主动更新（写 DB 后写 cache）
  - 仅 TTL（最终一致）

经验：
- “删缓存”通常比“更新缓存”更安全（避免把旧值写回）。
- TTL 要加抖动（jitter）避免同一时间大量 key 同时过期（雪崩）。


## 知识点 5：三大故障：穿透/击穿/雪崩 与对策

- 穿透：查不存在的数据 -> 每次都 miss 打到 DB
  - 对策：参数校验、布隆过滤器、空值缓存（negative caching）

- 击穿：某热点 key 过期瞬间，大量并发一起回源
  - 对策：互斥锁（singleflight）、提前刷新、逻辑过期

- 雪崩：大量 key 同时过期/缓存挂了 -> DB 被打爆
  - 对策：TTL 抖动、限流/降级、多级缓存、熔断


## 知识点 6：可运行：实现一个 TTLCache（最小）

下面实现：
- set/get，带过期时间
- 过期后自动 miss

注意：这是教学版，不包含 LRU 淘汰与并发安全。


In [ ]:
import time
from dataclasses import dataclass


@dataclass
class Entry:
    value: object
    expires_at: float


class TTLCache:
    def __init__(self):
        self._data = {}

    def set(self, key, value, ttl_sec: float):
        self._data[key] = Entry(value=value, expires_at=time.time() + ttl_sec)

    def get(self, key):
        e = self._data.get(key)
        if not e:
            return None
        if time.time() >= e.expires_at:
            self._data.pop(key, None)
            return None
        return e.value


c = TTLCache()
c.set('k', {'v': 1}, ttl_sec=0.2)
print('t0', c.get('k'))
time.sleep(0.25)
print('t1', c.get('k'))


## 知识点 7：cache-aside 示例：读回填 + 写删缓存

用一个“慢查询”模拟 DB：
- miss 时回源并 set
- 写入后删除 key，避免脏读


In [ ]:
import time


def db_get_user(user_id: int):
    time.sleep(0.05)  # 模拟慢
    return {'id': user_id, 'name': f'user{user_id}', 'updated_at': int(time.time())}


def cache_aside_get(cache, key, loader, ttl=1.0):
    v = cache.get(key)
    if v is not None:
        return v, 'hit'
    v = loader()
    cache.set(key, v, ttl_sec=ttl)
    return v, 'miss'


# reuse TTLCache from previous cell
cache = TTLCache()

v, tag = cache_aside_get(cache, 'user:1', lambda: db_get_user(1))
print(tag, v)
v, tag = cache_aside_get(cache, 'user:1', lambda: db_get_user(1))
print(tag, v)

# 写入（模拟）：先写 DB，再删缓存
cache._data.pop('user:1', None)
v, tag = cache_aside_get(cache, 'user:1', lambda: db_get_user(1))
print('after write', tag, v)


## 知识点 8：击穿防护：singleflight（互斥回源）

当热点 key 过期，大量并发回源会把 DB 打爆。

解决：同一 key 的回源只允许一个线程执行，其余等待结果（singleflight）。
下面用线程 + Lock 演示“有无互斥”的差异。


In [ ]:
import threading
import time


class SingleFlight:
    def __init__(self):
        self._locks = {}
        self._global = threading.Lock()

    def lock_for(self, key):
        with self._global:
            self._locks.setdefault(key, threading.Lock())
            return self._locks[key]


calls = 0
calls_lock = threading.Lock()

def slow_load():
    global calls
    time.sleep(0.05)
    with calls_lock:
        calls += 1
    return {'value': 'data', 't': time.time()}


def get_without_sf(cache, key):
    v = cache.get(key)
    if v is not None:
        return v
    v = slow_load()
    cache.set(key, v, ttl_sec=0.2)
    return v


def get_with_sf(cache, key, sf: SingleFlight):
    v = cache.get(key)
    if v is not None:
        return v
    lk = sf.lock_for(key)
    with lk:
        v = cache.get(key)
        if v is not None:
            return v
        v = slow_load()
        cache.set(key, v, ttl_sec=0.2)
        return v


def run(getter):
    global calls
    calls = 0
    cache = TTLCache()
    sf = SingleFlight()

    # 预热一次
    getter(cache, 'hot', sf) if getter.__name__ == 'get_with_sf' else getter(cache, 'hot')
    time.sleep(0.25)  # 让它过期

    ts = []
    for _ in range(20):
        if getter.__name__ == 'get_with_sf':
            t = threading.Thread(target=lambda: getter(cache, 'hot', sf))
        else:
            t = threading.Thread(target=lambda: getter(cache, 'hot'))
        ts.append(t)
    for t in ts:
        t.start()
    for t in ts:
        t.join()
    return calls


print('calls without sf:', run(get_without_sf))
print('calls with sf:', run(get_with_sf))


## 常见坑

- 把缓存当“强一致存储”：没有失效策略导致长期脏读
- key 设计不稳定：参数顺序不同导致缓存碎片
- TTL 不加抖动：同一时刻大量过期引发雪崩
- 击穿不做互斥：热点过期瞬间打爆 DB
- 空值不缓存：穿透导致 DB 被无意义请求持续攻击


## 综合小案例：为 REST API 增加“缓存层”（cache-aside）

以 GET /resource 为例：
- key：`v1:resource:{id}`
- miss：查 DB -> set cache (TTL + jitter)
- 写：写 DB -> 删除 cache
- 失败：缓存故障不影响主流程（降级到 DB）

建议输出一份“缓存策略文档”：key、TTL、失效方式、容量、命中率指标与告警。


In [ ]:
# 建议把缓存策略写成结构化文档（Key/TTL/失效/一致性/指标）。
# 本 cell 不运行代码。


## 自测题（不写代码也能回答）

- cache-aside 的读写流程分别是什么？为什么“写后删缓存”更常见？
- 穿透/击穿/雪崩分别是什么？典型对策有哪些？
- 为什么 TTL 要加抖动？
- 什么情况下不应该加缓存？（例如强一致、低命中、计算便宜）


## 练习题（建议写代码）

- 给 TTLCache 增加 LRU 淘汰（限制最大条目数）。
- 实现 negative caching：对不存在的 key 缓存一个短 TTL 的空值。
- 实现“逻辑过期”：过期后先返回旧值，同时后台刷新新值（了解）。
